In [120]:
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException

# Webdriver'ı başlat
driver = webdriver.Chrome()

# Oturum açma URL'si
login_url = 'https://b2b.hurtel.com/de/signin.html'

# Kullanıcı adı ve şifre
username = 'info3'
password = '123456'

try:
    # Oturum açma sayfasını aç
    driver.get(login_url)

    # Kullanıcı adı ve şifre giriş alanlarını bekle
    username_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'user_login')))
    password_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'user_pass')))


    # Kullanıcı adı ve şifreyi gir
    username_input.send_keys(username)
    password_input.send_keys(password, Keys.RETURN)  # Return tuşuna basarak giriş yap

except TimeoutException:
    print("Timeout hatası: Kullanıcı adı veya şifre giriş alanı bulunamadı.")
    driver.quit()
    exit()


#for url in urls:

# Oturum açma işlemi tamamlandıktan sonra fiyatları çek
#url = 'https://b2b.hurtel.com/de/products/baseus-metal-gleam-4in1-multifunktions-hub-usb-typ-c-usb-typ-c-stromversorgung-100-w-hdmi-4k-30-hz-1x-usb-3-2-gen-1-1x-usb-2-0-cahub-cy0g-69043.html?query_id=1'
#driver.get(url)

# Farklı URL'ler
url_list = [
    'https://b2b.hurtel.com/de/products/baseus-metal-gleam-4in1-multifunktions-hub-usb-typ-c-usb-typ-c-stromversorgung-100-w-hdmi-4k-30-hz-1x-usb-3-2-gen-1-1x-usb-2-0-cahub-cy0g-69043.html?query_id=1',
    'https://b2b.hurtel.com/de/products/baseus-superior-kabel-usb-lightning-2-4a-1-5-m-weiss-calys-b02-69054.html?query_id=1',
    # Ek URL'leri buraya ekleyin
]

# Scraping sonuçları için boş bir liste
scraped_data = []



# Oturum açma işlemi tamamlandıktan sonra fiyatları çek
for url in url_list:
    driver.get(url)

    # Sayfanın kaynak kodunu al
    page_source = driver.page_source



# BeautifulSoup ile HTML içeriğini analiz et
soup = BeautifulSoup(page_source, 'html.parser')

# GTIN değerini çek
ean_element = soup.find('span', class_='dictionary__name_txt', string='EAN')
if ean_element:
    ean_value = ean_element.find_next('span', class_='dictionary__value_txt').text.strip()
    print("GTIN:", ean_value)
else:
    print("GTIN değeri bulunamadı.")

# HAN değerini çek
produzenten_code_element = soup.find('span', class_='dictionary__name_txt', string='Produzenten-Code')
if produzenten_code_element:
    produzenten_code_value = produzenten_code_element.find_next('span', class_='dictionary__value_txt').text.strip()
    print("HAN:", produzenten_code_value)
else:
    print("HAN değeri bulunamadı.")

# Ürün adını çek
product_name_element = soup.find('h1', class_='product_name__name')
if product_name_element:
    product_name = product_name_element.text.strip()
    print("Artikelname:", product_name)
else:
    print("Ürün adı bulunamadı.")

# Ürün açıklamasını çek
beschreibung_section = soup.find('section', {'id': 'projector_longdescription'})
if beschreibung_section:
    # div etiketini kaldır
    div_to_remove = beschreibung_section.find('div', {'style': 'max-width: 950px; margin: 0 auto'})
    if div_to_remove:
        div_to_remove.extract()
        
    print("Beschreibung:", beschreibung_section.prettify())


# Fiyatı çek
price_element = soup.find('strong', {'id': 'projector_price_value'})
if price_element:
    price_text = price_element.text.strip().split(' ')[0]
    print("Durchschnittlicher Einkaufspreis (netto):", price_text)
else:
    print("Fiyat bulunamadı.")

figure_elements = soup.find_all('figure', class_='photos__figure')

# Sadece ilk 5 figure elementini ele al
for i, figure_element in enumerate(figure_elements[:5]):
    a_element = figure_element.find('a', class_='photos__link')
    
    # a elementinin href özelliğini yazdır
    if a_element:
        href_value = a_element.get('href')
        print(f"Bild {i + 1} Pfad/URL: {href_value}")
    else:
        print(f"Bild {i + 1} için href bulunamadı.")

# Bullet Point bölümü bul
long_description_section = soup.find('section', {'id': 'projector_longdescription'})

if long_description_section:
    # Tüm <ul> etiketlerini seç
    ul_elements = long_description_section.find_all('ul')

    if ul_elements:
        # Son <ul> etiketini al
        last_ul_element = ul_elements[-1]

        # Son <ul> etiketinin içindeki <li> öğelerini al
        li_elements = last_ul_element.find_all('li')

        # İlk 5 öğeyi yazdır
        for i, li_element in enumerate(li_elements[:5], start=1):
            print(f"Değer {i}: {li_element.text.strip()}")
    else:
        print("<ul> etiketi bulunamadı.")
else:
    print("projector_longdescription id'sine sahip section bulunamadı.")


scraped_data.append({"GTIN": ean_value, "HAN": produzenten_code_value, "Artikelname": product_name, "Beschreibung": beschreibung_section.get_text(strip=True, separator=' '), "Price": price_text})

csv_file_path = "scraped_data.csv"


with open(csv_file_path, mode="w", encoding="utf-8", newline="") as csv_file:
    fieldnames = ["GTIN", "HAN", "Artikelname", "Beschreibung", "Price"]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for data in scraped_data:
        writer.writerow(data)


# Tarayıcıyı kapat
driver.quit()

#SADECE FIYAT YÖNTEMI DOGRU


GTIN: 6953156205444
HAN: CALYS-B02
Artikelname: Baseus Superior Kabel USB - Lightning 2,4A 1,5 m Weiß (CALYS-B02)
Beschreibung: <section class="longdescription cm col-12" data-dictionary="true" id="projector_longdescription">
 <p>
  <strong>
   Baseus Superior:
  </strong>
 </p>
 <ul>
  <li>
   Long charging USB / Lightning cable
  </li>
  <li>
   <strong>
    Fast and safe charging
   </strong>
   –
   <strong>
    2.4A
   </strong>
   current and
   <strong>
    bold copper wire
   </strong>
   provide a faster and more secure charging.
  </li>
  <li>
   <strong>
    Reinforced joint
   </strong>
   – With extra joint reinforcement accessory got a
   <strong>
    truly anti-break design
   </strong>
   .
  </li>
  <li>
   <strong>
    Strong plug shell
   </strong>
   – High-quality plug,
   <strong>
    resistant to both oxidation and corrosion
   </strong>
   .
  </li>
  <li>
   <strong>
    Fabric braid
   </strong>
   – Elastic cord was braided by strong and durable textile. High

In [3]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException
import os
import pandas as pd
import urllib.parse
import urllib.request
from tqdm import tqdm  # tqdm is used for progress bar

current_directory = os.getcwd()
print("Current Directory:", current_directory)

chrome_options = Options()
chrome_options.add_argument("--headless")  # Tarayıcıyı görünmez modda başlatmak için

driver = webdriver.Chrome(options=chrome_options)

# Oturum açma URL'si
login_url = 'https://b2b.hurtel.com/de/signin.html'

# Kullanıcı adı ve şifre
username = 'info3'
password = '123456'

try:
    # Oturum açma sayfasını aç
    driver.get(login_url)

    # Kullanıcı adı ve şifre giriş alanlarını bekle
    username_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'user_login')))
    password_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'user_pass')))

    # Kullanıcı adı ve şifreyi gir
    username_input.send_keys(username)
    password_input.send_keys(password, Keys.RETURN)  # Return tuşuna basarak giriş yap

    # Farklı URL'ler
    url_list = [
        'https://b2b.hurtel.com/de/products/baseus-metal-gleam-4in1-multifunktions-hub-usb-typ-c-usb-typ-c-stromversorgung-100-w-hdmi-4k-30-hz-1x-usb-3-2-gen-1-1x-usb-2-0-cahub-cy0g-69043.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-superior-kabel-usb-lightning-2-4a-1-5-m-weiss-calys-b02-69054.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-magnetischer-autotelefonhalter-kabelloses-qi-ladegerat-15-w-magsafe-kompatibel-fur-iphone-schwarz-wxjn-01-70582.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-super-si-1c-schnellladegerat-usb-typ-c-30w-power-delivery-quick-charge-schwarz-ccsup-j01-72252.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-metal-gleam-series-5-in-1-multifunktionale-typ-c-hub-dockingstation-grau-30hz-version-grau-81911.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-gan3-pro-schnellladegerat-gan-2-x-usb-typ-c-2-x-usb-65w-pd-qc4-0-afc-pps-kabel-usb-typ-c-usb-typ-c-1m-schwarz-ccgp040101-88811.html?query_id=1',
        'https://b2b.hurtel.com/de/products/bluetooth-sender-autoladegerat-baseus-s-13-overseas-edition-schwarz-149092.html?query_id=1',
        'https://b2b.hurtel.com/de/products/gan-2xusb-c-1xusb-65w-schnellladegerat-baseus-gan5-pro-usb-c-usb-c-100w-kabel-schwarz-150738.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-gan2-pro-schnellladegerat-100w-usb-usb-typ-c-quick-charge-4-power-delivery-schwarz-ccgan2p-l01-69149.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-gan2-pro-schnellladegerat-100w-usb-usb-typ-c-quick-charge-4-power-delivery-schwarz-ccgan2p-l01-69149.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-encok-h17-3-5-mm-kopfhorer-mit-miniklinke-weiss-ngcr020002-92254.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-blade-ultradunne-powerbank-100w-20000mah-schwarz-pd-qc-scp-fcp-ppdgl-01-95324.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cube-pro-65w-gan-ladegerat-2x-usb-c-usb-a-schwarz-152681.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-cable-durabel-kabel-mit-nylon-geflochtenes-ladekabel-usb-lightning-qc3-0-1-5a-2m-rot-calklf-c09-46809.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-superme-szybka-ladowarka-samochodowa-usb-usb-typ-c-100w-pps-schnellladung-power-delivery-czarny-cczx-01-71974.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-super-si-1c-schnellladegerat-usb-typ-c-30w-power-delivery-quick-charge-weiss-ccsup-j02-72253.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-superior-kabel-usb-micro-usb-zum-schnellladen-2a-1m-weiss-camys-02-72561.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-bowie-e2-tws-bluetooth-5-2-wireless-kopfhorer-wasserdicht-ip55-schwarz-ngtw090001-84968.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-glimmer-series-schnellladekabel-usb-c-480-mbit-s-pd-100-w-1-m-schwarz-135438.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-glimmer-series-schnellladekabel-usb-c-lightning-480-mbit-s-pd-20-w-1-m-schwarz-135453.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-l54-kopfhoreradapter-usb-c-auf-3-5-mm-audiobuchse-dac-24-bit-48-khz-schwarz-catl54-01-53137.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-super-si-1c-schnellladegerat-usb-typ-c-20w-power-delivery-usb-kabel-typ-c-lightning-1m-schwarz-tzccsup-b01-66709.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-superior-kabel-usb-lightning-2-4a-2-m-weiss-calys-c02-69056.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-a3-akku-autostaubsauger-135-w-15000-pa-schwarz-crxcqa3-0a-70214.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-enjoy-fm-transmitter-autoladegerat-led-2x-usb-3-5mm-klinke-kabellos-mp3-player-bluetooth-5-0-3-4a-schwarz-cclh-01-72566.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-e3-wireless-bluetooth-5-0-tws-ohrhorer-wasserdicht-ip64-weiss-ngtw080002-76976.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-super-si-1c-fast-wall-charger-usb-type-c-25w-power-delivery-quick-charge-white-ccsp020102-81910.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-lite-series-multifunktionaler-hub-usb-typ-c-2-x-usb-3-0-usb-typ-c-hdmi-1-4-sd-tf-schwarz-wkqx050001-92250.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-tungsten-gold-kabel-usb-a-usb-c-480mb-s-100w-1m-schwarz-cawj000001-121053.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-bowie-h2-anc-kabellose-on-ear-kopfhorer-grau-138565.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-seashell-series-verstellbarer-telefonstander-mit-spiegel-lila-148578.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-backseat-car-mount-auto-rucksitz-kopfstutze-tablet-smartphone-halterung-4-7-12-9-schwarz-suhz-01-27418.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-future-gravity-car-mount-auto-halter-luftungsgitter-halterung-gravity-automatischer-schwarz-suyl-wl01-44609.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-cable-durabel-kabel-mit-nylon-geflochtenes-ladekabel-usb-usb-c-qc3-0-3a-1m-schwarz-rot-catklf-b91-46797.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-cable-durabel-kabel-mit-nylon-geflochtenes-ladekabel-usb-usb-c-qc3-0-2a-2m-rot-catklf-c09-46798.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-audio-converter-l46-adapter-splitter-von-lightning-auf-2x-lightning-ladeanschluss-schwarz-cal46-01-46954.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-backseat-vehicle-phone-holder-hook-auto-rucksitz-kopfstutze-tablet-smartphone-halterung-4-0-6-5-schwarz-suhz-a01-49704.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-magic-series-dual-qc-quick-charge-3-0-2x-usb-45w-6a-autoladegerat-schwarz-ccmlc20a-01-50575.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-rainbow-circle-klettbander-klettband-klett-kabelorganizer-1m-schwarz-acmgt-e01-50591.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-kabel-usb-kabel-4in1-lightning-2x-usb-typ-c-micro-usb-kabel-mit-nylon-geflochtenes-3-5a-1-2m-schwarz-ca1t4-b01-51044.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-selbstklebende-kartentasche-fur-handy-grau-ackd-b0g-51718.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-2-in-1-auto-sicherheit-fenster-hammer-sicherheitsgurt-messer-grau-crsfh-0g-53294.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-autoladegerat-2x-usb-4-8a-24w-mit-lcd-grau-ccbx-0g-56057.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-ultradunner-selbstklebender-ringhalter-telefonstander-rosa-suyb-0r-60574.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-2x-kabel-schnellladen-ladekabel-datenkabel-usb-lightning-power-delivery-2-4-a-1-5-m-weiss-tzcalzj-02-63961.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-usb-type-c-usb-type-c-kabel-schnellladung-power-delivery-quick-charge-100-w-5-a-1-m-schwarz-catwj-01-64393.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-golden-contactor-pro-quick-charger-auto-ladegerat-2x-usb-40-w-quick-charge-scp-fcp-afc-usb-usb-typ-c-kabel-grau-tzccjd-a0g-67407.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-radar-magnetischer-autohalter-kompatibel-mit-magsafe-fur-iphone-fur-armaturenbrett-luftungsgitter-schwarz-suld-01-68042.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-golden-contactor-pro-schnelles-usb-autoladegerat-typ-c-usb-40-w-power-delivery-3-0-quick-charge-4-scp-fcp-afc-usb-kabel-usb-typ-c-grau-tzccjd-0g-68400.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-super-si-1c-schnellladegerat-usb-type-c-20w-power-delivery-usb-type-c-lightning-kabel-1m-blau-tzccsup-b03-71973.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-share-together-autoladegerat-3x-usb-usb-type-c-120w-pps-quick-charge-power-delivery-grau-ccbt-b0g-72078.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-share-together-autoladegerat-2x-usb-2x-usb-type-c-120w-pps-quick-charge-power-delivery-grau-ccbt-a0g-72257.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-powerbank-power-starter-jump-starter-startgerat-booster-20000mah-12v-2000a-1000a-krokodilklemmenkabel-schwarz-cgnl020001-76529.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-compact-schnellladegerat-2x-usb-usb-type-c-30w-3a-power-delivery-quick-charge-schwarz-ccxj-e01-76967.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-super-si-pro-schnellladegerat-usb-usb-type-c-30w-power-delivery-quick-charge-schwarz-ccsupp-e01-77055.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-super-si-pro-usb-usb-typ-c-schnellladegerat-30w-power-delivery-schnellladung-weiss-ccsupp-e02-77056.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-crystal-shine-series-kabel-usb-kabel-fur-schnelles-aufladen-und-datenubertragung-usb-typ-a-lightning-2-4a-1-2m-blau-cajy000003-81328.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-encok-c17-kabelgebundener-in-ear-kopfhorer-mit-usb-typ-c-mikrofon-weiss-ngcr010002-84967.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-selbstklebender-faltbarer-telefonstander-schwarz-luxz000001-87872.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-metal-age-ii-gravitations-autotelefonhalter-fur-luftungsgitter-silber-sujs000012-90964.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-lite-series-multifunktionaler-hub-usb-typ-c-2-x-usb-3-0-usb-typ-c-pd-hdmi-1-4-sd-tf-schwarz-wkqx050101-92252.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-auto-pflegespray-zur-reinigung-des-autoinnenraums-300ml-weiss-cryh000002-95546.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-golden-contactor-pro-autoladegerat-usb-a-2x-usb-c-65w-qc4-0-blau-cgjp010003-96888.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-i-wok2-led-lampe-fur-desktop-bildschirmbeleuchtung-schwarz-dgiw000101-97137.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-storm-3-wireless-bluetooth-5-2-tws-kopfhorer-mit-anc-schwarz-ngtw140101-97228.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-storm-1-wireless-bluetooth-5-2-tws-in-ear-kopfhorer-mit-anc-enc-weiss-ngtw140202-97231.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-digital-led-display-dual-wireless-charger-mit-digitalanzeige-20w-schwarz-wxsx010101-97232.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-adaman2-powerbank-mit-digitalanzeige-10000mah-30w-2-x-usb-1x-usb-type-c-overseas-edition-pd-qc-scp-schwarz-ppad040101-106511.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-wireless-active-stylus-austauschbare-spitze-weiss-sxbc020002-106849.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-mvp-2-winkelkabel-power-delivery-kabel-mit-seitlichem-usb-type-c-lightning-stecker-1m-20w-blau-cavp000221-107398.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-stylus-mit-kabellosem-laden-fur-ipad-weiss-austauschbare-spitze-107610.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-smooth-writing-active-stylus-eingabestift-mit-led-anzeige-fur-ipad-weiss-usb-c-stromkabel-3a-0-3m-und-austauschbarer-spitze-active-sxbc000202-107611.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-kabelloses-autoladegerat-mit-magsafe-15w-halterung-fur-luftungsgitter-usb-typ-c-25w-weiss-cw01-135773.html?query_id=1',
        'https://b2b.hurtel.com/de/products/kameraglas-fur-iphone-13-pro-max-13-pro-baseus-kameraglas-137924.html?query_id=1',
        'https://b2b.hurtel.com/de/products/verlangerungskabel-usb-2-0-0-5-m-baseus-airjoy-serie-schwarz-146054.html?query_id=1',
        'https://b2b.hurtel.com/de/products/usb-c-schnelllade-datenubertragungskabel-lightning-pd-20-w-1-m-baseus-superior-series-lila-146056.html?query_id=1',
        'https://b2b.hurtel.com/de/products/usb-c-kabel-lightning-20-w-pd-1-m-baseus-superior-serie-blau-146057.html?query_id=1',
        'https://b2b.hurtel.com/de/products/kabel-usb-c-usb-c-480-mbit-s-240-w-3-m-baseus-tungsten-gold-cawj040101-schwarz-147390.html?query_id=1',
        'https://b2b.hurtel.com/de/products/verstellbarer-telefonstander-der-baseus-seashell-serie-rosa-148575.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-explorer-series-kabel-usb-lightning-2-4a-1-m-weiss-cats010002-148587.html?query_id=1',
        'https://b2b.hurtel.com/de/products/bluetooth-sender-autoladegerat-baseus-s-09a-overseas-edition-schwarz-149094.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-bowie-ez10-tws-bluetooth-5-3-kabellose-kopfhorer-weiss-149245.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-neogravity-magnetische-autohalterung-fur-cockpit-luftungsschlitze-overseas-edition-schwarz-150009.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-t-space-series-usb-hub-fur-tesla-model-3-y-mit-integriertem-45-w-usb-c-kabel-schwarz-150035.html?query_id=1',
        'https://b2b.hurtel.com/de/products/stylus-hulle-fur-das-baseus-smooth-writing-2-tablet-weiss-151802.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-brilliance-series-tastaturhulle-fur-ipad-10-9-2022-10-generation-usb-c-kabel-schwarz-152141.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-smooth-writing-2-stylus-mit-aktiver-spitze-fur-ipad-mit-usb-c-kabel-und-austauschbarer-spitze-weiss-152544.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-smooth-writing-2-stylus-mit-aktiver-spitze-fur-ipad-mit-austauschbarer-spitze-weiss-152545.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-smooth-writing-2-stylus-mit-aktiver-spitze-fur-ipad-usb-a-lightning-kabel-und-austauschbare-spitze-weiss-152546.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-smooth-writing-2-overseas-edition-stylus-mit-aktiver-spitze-fur-ipad-mit-austauschbarer-spitze-weiss-155489.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-adapter-von-usb-type-c-auf-usb-kabel-splitter-schwarz-caaotg-01-26193.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-typ-c-kabel-300cm-grau-schwarz-51808.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-rainbow-circle-klettbander-klettverschluss-klettband-kabelorganisator-3m-schwarz-acmgt-f01-50596.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-charging-quick-charger-wandladegerat-eu-usb-quick-charge-3-0-qc-3-0-weiss-ccall-bx02-40777.html?query_id=1',
        'https://b2b.hurtel.com/de/products/magnetische-halterung-fur-luftungsschlitze-von-baseus-overseas-edition-schwarz-150200.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-filterset-fur-smart-feeder-8-stk-weiss-acly010002-106851.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-comfortride-series-auto-kopfstutzenkissen-mit-2-materialien-dunkelgrun-154597.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-comfortride-series-auto-kopfstutzenkissen-mit-2-materialien-beige-154598.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-t-space-series-auto-mittelkonsolen-armlehnenbox-twilight-grey-155479.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-autotelefonhalter-fur-luftungsschlitz-schwarz-sugp-01-63419.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-adapter-von-usb-auf-usb-type-c-kabel-splitter-otg-schwarz-catotg-01-25587.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-ba04-mini-bluetooth-5-0-usb-adapter-empfanger-sender-fur-computer-schwarz-zjba000001-87871.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-gehartetes-glas-honor-x30-0-3-mm-klar-2-stuck-sgbl024302-107614.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-bipow-schnelllade-powerbank-20000mah-20w-schwarz-overseas-edition-usb-a-micro-usb-kabel-0-25m-schwarz-ppbd050501-121174.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-auto-halter-armaturenbrett-windschutzscheibe-halterung-gravity-automatischer-schwarz-suyl-tk01-62129.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-metal-age-gravity-autohalterung-metall-gravity-autohalterung-fur-cd-steckplatz-schwarz-suyl-j01-49697.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-gravity-car-mount-4-6-schwerkraft-autohalterung-fur-telefonlufter-schwarz-suyl-01-48211.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-ba01-bluetooth-5-0-audioempfangerkabel-aux-klinke-audioadapter-schwarz-caba01-01-49476.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-cable-strapazierfahiges-nylonkabel-usb-usb-c-qc3-0-2a-3m-rot-catklf-u09-51807.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-audio-converter-l41-adapter-von-usb-c-auf-usb-c-anschluss-3-5-mm-kopfhoreranschluss-schwarz-catl41-01-46846.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-circular-smart-mini-auto-ladegerat-autoadapter-2x-usb-qc3-0-quick-charge-3-0-scp-afc-30w-schwarz-ccall-yd01-46978.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-2x-usb-kabel-usb-typ-c-schnellladung-power-delivery-quick-charge-40-w-5-a-1-5-m-weiss-tzcatzj-02-63962.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-crystal-shine-series-kabel-usb-kabel-fur-schnelles-aufladen-und-datenubertragung-usb-typ-c-usb-typ-c-100w-2m-schwarz-cajy000701-84964.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-kabel-strapazierfahiges-nylonkabel-usb-usb-c-qc3-0-2a-2m-schwarz-rot-catklf-c91-46800.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-wisdom-induktionsladegerat-autotelefonhalter-schwarz-cgzx000001-90966.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-einfaches-flachkabel-usb-usb-type-c-supercharge-5a-40w-quick-charge-3-0-qc-3-0-23cm-grau-catmbj-bg1-50851.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-ingenuity-series-steckeradapter-usb-typ-c-auf-usb-a-3-2-gen-1-schwarz-zjjq000001-92270.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-cable-durabel-kabel-mit-nylon-geflochtenes-ladekabel-usb-micro-usb-2a-3m-schwarz-grau-camklf-hg1-51916.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-2-wege-switch-splitter-hdmi-umschalter-2x-hdmi-4k-30-hz-grau-cahub-bc0g-60019.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-compact-ladegerat-2x-usb-10-5w-schwarz-ccxj010201-93149.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-bs-w530-15w-induktionsladegerat-usb-c-kabel-usb-c-24w-1m-schwarz-150136.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-small-ears-series-auto-halterung-magnet-handyhalter-fur-armaturenbrett-schwarz-suer-b01-22017.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-autoladegerat-splitter-2x-usb-3-1a-17w-2x-zigarettenanzunder-buchse-80w-schwarz-crdyq-01-52913.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-tungsten-kabel-usb-lightning-2-4-a-1-m-schwarz-calwj-01-68253.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-bipow-schnelllade-powerbank-20000mah-15w-schwarz-overseas-edition-usb-a-micro-usb-kabel-0-25m-schwarz-ppbd050101-120978.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-compact-ladegerat-3x-usb-17w-weiss-ccxj020102-93166.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-cable-strapazierfahiges-nylonkabel-usb-micro-usb-1-5a-2m-rot-camklf-c09-46789.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-kabel-strapazierfahiges-nylonkabel-usb-lightning-qc3-0-2-4a-0-5m-schwarz-grau-calklf-ag1-46802.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-superior-cable-usb-typ-c-lightning-power-delivery-20-w-1-m-weiss-catlys-a02-69051.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-crystal-shine-series-kabel-usb-kabel-fur-schnelles-aufladen-und-datenubertragung-usb-typ-c-lightning-20w-2m-schwarz-cajy000301-81336.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-kabel-strapazierfahiges-nylonkabel-usb-usb-c-qc3-0-3a-1m-rot-catklf-b09-46795.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-cable-strapazierfahiges-nylonkabel-usb-usb-c-qc3-0-2a-3m-schwarz-rot-catklf-u91-51809.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-magnet-iron-suit-2-selbstklebende-metallplattenabdeckungen-fur-magnetische-autohalterungen-silber-acdr-a0s-37942.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-cable-durabel-kabel-mit-nylon-geflochtenes-ladekabel-usb-c-pd-usb-c-pd-pd2-0-60w-20v-3a-qc3-0-2m-rot-catklf-h09-46965.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-kabel-hdmi-2-0-kabel-4k-60-hz-3d-hdr-18-gbps-1-m-schwarz-cakgq-a01-60575.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-superior-kabel-usb-usb-typ-c-6a-66w-2m-weiss-catys-a02-69059.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-crystal-shine-series-kabel-usb-kabel-fur-schnelles-aufladen-und-datenubertragung-usb-typ-c-usb-typ-c-100w-1-2m-schwarz-cajy000601-84961.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-metal-age-ii-gravitations-autotelefonhalter-fur-luftungsgitter-schwarz-sujs000001-90962.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-metal-age-ii-gravitations-autotelefonhalter-fur-luftungsgitter-dunkelgrau-sujs000013-90963.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-cable-strapazierfahiges-nylonkabel-usb-micro-usb-1-5a-2m-schwarz-rot-camklf-c91-46791.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-kabel-strapazierfahiges-nylonkabel-usb-usb-c-qc3-0-3a-0-5m-schwarz-rot-catklf-a91-46793.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-three-primary-colors-3in1-usb-kabel-micro-usb-lightning-usb-c-nylon-geflochten-3-5a-1-2m-schwarz-camlt-bsy01-48209.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-cable-strapazierfahiges-nylonkabel-usb-lightning-qc3-0-2a-3m-schwarz-rot-calklf-r91-51805.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-magnetic-bracket-power-bank-mit-magsafe-wireless-charging-10000mah-20w-overseas-edition-blau-ppcx000203-usb-type-c-baseus-xiaobai-series-60w-0-5m-107554.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-encok-h06-earphones-kopfhorer-earbud-ohrhorer-mit-fernbedienung-schwarz-ngh06-01-46837.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-kabel-usb-typ-c-supercharge-40w-quick-charge-3-0-qc-3-0-kabel-1m-grau-schwarz-catklf-pg1-50563.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-magic-series-pps-quick-charge-4-0-qc3-0-45w-6a-autoladegerat-schwarz-ccmlc20c-01-50572.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-smart-vehicle-bracket-wireless-charger-automatischer-elektrisch-kfz-halterung-kabelloses-qi-ladegerat-15w-schwarz-wxhw03-01-54483.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-jelly-qi-15-w-kabelloses-ladegerat-fur-kopfhorer-usb-kabel-usb-typ-c-schwarz-wxgd-01-61597.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-magnetische-led-nachttischlampe-fur-zuhause-kuche-zimmer-weiss-dgxc-02-63957.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-grain-car-charger-mini-autoladegerat-2x-usb-3-1a-schwarz-ccall-ml01-44600.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-kabel-strapazierfahiges-nylonkabel-usb-lightning-qc3-0-2-4a-1m-schwarz-gold-calklf-bv1-46808.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-full-view-blind-spot-mirror-2x-toter-winkel-spiegel-weitwinkel-spiegel-schwarz-acmdj-01-46953.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-nimble-flachkabel-usb-usb-c-kabel-mit-halter-2a-0-23m-schwarz-catmbj-01-48219.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-regeneratorreiniger-autowischerscharfer-messer-messer-silber-crxfq-0s-60869.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-high-definition-series-hdmi-8k-to-hdmi-8k-adapter-cable-1-5m-black-107403.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-ultracontrol-lite-series-autotelefonhalter-schwarz-149259.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-bullet-an-on-board-magnetic-bracket-auto-halterung-magnet-handyhalter-fur-armaturenbrett-schwarz-suyzd-01-40774.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-square-pps-smart-mini-auto-ladegerat-autoadapter-usb-quick-charge-4-0-qc-4-0-und-usb-c-pd-3-0-scp-schwarz-ccall-as01-44602.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-cable-durabel-kabel-mit-nylon-geflochtenes-ladekabel-usb-micro-usb-qc3-0-2-4a-1m-rot-camklf-b09-46786.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-cable-durabel-kabel-mit-nylon-geflochtenes-ladekabel-usb-lightning-qc3-0-2-4a-0-5m-rot-calklf-a09-46801.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-mini-auto-abfallbehalter-mit-deckel-schwarz-crljt-01-52907.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-metal-age-ii-gravitations-autotelefonhalter-fur-luftungsgitter-grun-sujs000006-90965.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-qpow-digital-display-powerbank-mit-schnellladung-20000-mah-20-w-qc-pd-scp-fcp-mit-integriertem-lightning-kabel-schwarz-135427.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-qpow-digital-display-powerbank-mit-schnellladung-20000-mah-22-5-w-qc-pd-scp-fcp-mit-integriertem-usb-c-kabel-weiss-135430.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-bsba-02-overseas-edition-bluetooth-aux-sender-schwarz-149093.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-osculum-gravitations-autohalterung-fur-windschutzscheibe-oder-armaturenbrett-37948.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-cable-durabel-kabel-mit-nylon-geflochtenes-ladekabel-usb-usb-c-qc3-0-3a-0-5m-rot-catklf-a09-46792.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-kabel-strapazierfahiges-nylonkabel-usb-lightning-qc3-0-2-4a-1m-schwarz-rot-calklf-b19-46807.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-kabel-strapazierfahiges-nylonkabel-usb-c-pd-usb-c-pd-pd2-0-60w-20v-3a-qc3-0-1m-rot-catklf-g09-46962.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-heyo-countdown-timer-schwarz-acdjs-01-56252.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-auto-halter-armaturenbrett-windschutzscheibe-halterung-gravity-automatischer-schwarz-suyl-jy01-62912.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-usb-type-c-kabel-lightning-fast-charging-power-delivery-20-w-2-m-schwarz-catlwj-a01-64314.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-superior-kabel-usb-usb-typ-c-6a-66w-1m-weiss-catys-02-69058.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-crystal-shine-series-kabel-usb-kabel-fur-schnelles-aufladen-und-datenubertragung-usb-typ-c-lightning-20w-2m-blau-cajy000303-81337.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-crystal-shine-series-kabel-usb-kabel-fur-schnelles-aufladen-und-datenubertragung-usb-typ-c-usb-typ-c-100w-2m-blau-cajy000703-84965.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-yiven-m30-stereo-audiokabel-aux-3-5-mm-mini-klinkenstecker-1-5-m-silber-schwarz-cam30-cs1-106590.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-metal-ageii-schwerkrafthalterung-fur-runden-auto-luftungsschlitz-dunkelgrau-sujs030013-121183.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-glimmer-series-schnellladekabel-usb-a-usb-c-100-w-480-mbit-s-2-m-schwarz-135434.html?query_id=1',
        'https://b2b.hurtel.com/de/products/3in1-usb-micro-usb-lightning-usb-c-3-5a-1-2m-kabel-baseus-starspeed-schwarz-135765.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-wireless-charger-gravity-auto-halter-air-vent-mount-handy-stander-qi-induktive-schnellladestation-schwarz-wxyl-01-37966.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-mvp-elbow-micro-usb-doppelseitig-kabel-mit-l-form-stecker-2m-1-5a-schwarz-cammvp-b01-41697.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-cable-durabel-kabel-mit-nylon-geflochtenes-ladekabel-usb-micro-usb-qc3-0-2-4a-1m-schwarz-rot-camklf-b91-46788.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-cable-strapazierfahiges-nylonkabel-usb-micro-usb-1-5a-2m-schwarz-grau-camklf-cg1-46790.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-kabel-strapazierfahiges-nylonkabel-usb-lightning-qc3-0-2-4a-1m-rot-calklf-b09-46805.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-cable-durabel-kabel-mit-nylon-geflochtenes-ladekabel-usb-lightning-qc3-0-1-5a-2m-schwarz-golden-calklf-cv1-46812.html?query_id=1',
        'https://b2b.hurtel.com/de/products/baseus-cafule-kabel-strapazierfahiges-nylonkabel-usb-c-pd-usb-c-pd-pd2-0-60w-20v-3a-qc3-0-2m-schwarz-grau-catklf-hg1-46967.html?query_id=1'             
]

    # Scraping sonuçları için boş bir liste
    scraped_data = []

    # Oturum açma işlemi tamamlandıktan sonra verileri çek
    # for url in url_list:
    for url in tqdm(url_list, desc="Scraping Progress"):
        driver.get(url)

        # Sayfanın kaynak kodunu al
        page_source = driver.page_source

        # BeautifulSoup ile HTML içeriğini analiz et
        soup = BeautifulSoup(page_source, 'html.parser')

        # GTIN değerini çek
        ean_element = soup.find('span', class_='dictionary__name_txt', string='EAN')
        if ean_element:
            ean_value = ean_element.find_next('span', class_='dictionary__value_txt').text.strip()
        else:
            ean_value = "GTIN degeri bulunamadi."

        # HAN değerini çek
        produzenten_code_element = soup.find('span', class_='dictionary__name_txt', string='Produzenten-Code')
        if produzenten_code_element:
            produzenten_code_value = produzenten_code_element.find_next('span', class_='dictionary__value_txt').text.strip()
        else:
            produzenten_code_value = "HAN degeri bulunamadi."

        # Ürün adını çek
        product_name_element = soup.find('h1', class_='product_name__name')
        if product_name_element:
            product_name = product_name_element.text.strip()
        else:
            product_name = "Urun adi bulunamadi."

        # Ürün açıklamasını çek
        beschreibung_section = soup.find('section', {'id': 'projector_longdescription'})
        if beschreibung_section:
            # div etiketini kaldır
            div_to_remove = beschreibung_section.find('div', {'style': 'max-width: 950px; margin: 0 auto'})
            if div_to_remove:
                div_to_remove.extract()

            # Beschreibung içeriğini HTML olarak al
            beschreibung_html = beschreibung_section.prettify()
            
            # Alt satırları temizle
            beschreibung_text = beschreibung_html.replace('\n', ' ')
        else:
            beschreibung_text = "Urun Aciklamasi bulunamadi."

        # Fiyatı çek
        price_element = soup.find('strong', {'id': 'projector_price_value'})
        if price_element:
            price_text = price_element.text.strip().split(' ')[0]
        else:
            price_text = "Fiyat bulunamadi."

        # Resimleri Çek
        figure_elements = soup.find_all('figure', class_='photos__figure')
        image_urls = [figure_element.find('a', class_='photos__link').get('href') if figure_element.find('a', class_='photos__link') else "" for figure_element in figure_elements[:5]]

        # Sadece ilk 5 figure elementini ele al
        image_paths = [f"Bild {i + 1} Pfad/URL: {image_urls[i]}" if i < len(image_urls) else f"Bild {i + 1} için href bulunamadı." for i in range(5)]

        # Bullet Point bölümü bul
        long_description_section = soup.find('section', {'id': 'projector_longdescription'})

        if long_description_section:
            # Tüm <ul> etiketlerini seç
            ul_elements = long_description_section.find_all('ul')

            if ul_elements:
                # Son <ul> etiketini al
                last_ul_element = ul_elements[-1]

                # Son <ul> etiketinin içindeki <li> öğelerini al
                li_elements = last_ul_element.find_all('li')

                # İlk 5 öğeyi bir liste olarak sakla
                bullet_points = [li_element.text.strip() for li_element in li_elements[:5]]



        # Sonuçları scraped_data listesine ekle
        scraped_data.append({
            "GTIN": ean_value,
            "HAN": produzenten_code_value,
            "Artikelname": product_name,
            "Beschreibung": beschreibung_text,
            "Durchschnittlicher Einkaufspreis (netto)": price_text,
            "Bild 1 Pfad/URL": image_urls[0] if image_urls else "",
            "Bild 2 Pfad/URL": image_urls[1] if len(image_urls) > 1 else "",
            "Bild 3 Pfad/URL": image_urls[2] if len(image_urls) > 2 else "",
            "Bild 4 Pfad/URL": image_urls[3] if len(image_urls) > 3 else "",
            "Bild 5 Pfad/URL": image_urls[4] if len(image_urls) > 4 else "",
            "Bullet Point 1": bullet_points[0] if bullet_points else "",
            "Bullet Point 2": bullet_points[1] if len(bullet_points) > 1 else "",
            "Bullet Point 3": bullet_points[2] if len(bullet_points) > 2 else "",
            "Bullet Point 4": bullet_points[3] if len(bullet_points) > 3 else "",
            "Bullet Point 5": bullet_points[4] if len(bullet_points) > 4 else "", 
        })

        # İndirme işlemi için bir bekleme ekleyin
        time.sleep(2)  # Bu süreyi isteğinize göre ayarlayabilirsiniz

finally:
    # Tüm URL'leri ziyaret ettikten sonra CSV dosyasına yaz
    csv_file_path = "C:/Users/mubat/Desktop/MC/Murathan IM/py/scraped_data.csv"

    with open(csv_file_path, mode="w", encoding="utf-8-sig", newline="") as csv_file:
        fieldnames = ["GTIN", "HAN", "Artikelname", "Beschreibung", "Durchschnittlicher Einkaufspreis (netto)", "Bild 1 Pfad/URL", "Bild 2 Pfad/URL", "Bild 3 Pfad/URL", "Bild 4 Pfad/URL", "Bild 5 Pfad/URL", "Bullet Point 1", "Bullet Point 2", "Bullet Point 3", "Bullet Point 4", "Bullet Point 5"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames, delimiter='\t')  # Sekme karakterini kullanarak ayırma
        writer.writeheader()

        # URL'leri yaz
        for data in scraped_data:
            # Sadece belirli sütunları al ve birleştir
            combined_data = "\t".join([data[field] for field in fieldnames])
            writer.writerow({"GTIN": combined_data})

    # Tarayıcıyı kapat
    driver.quit()
#SON HALI, BASARILI, SUPER

Current Directory: C:\Users\mubat\Desktop\MC\Murathan IM\py


Scraping Progress: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 187/187 [12:53<00:00,  4.14s/it]


PermissionError: [Errno 13] Permission denied: 'C:/Users/mubat/Desktop/MC/Murathan IM/py/scraped_data.csv'

In [2]:
!/usr/bin/python3
# -*- coding: utf-8 -*-

import feedparser

"""
    cnn'den son dakika haberlerini çekme
"""
url = "http://www.cnnturk.com/feed/rss/news"
haberler = feedparser.parse(url)

i = 0
for haber in haberler.entries:
    i += 1
    print(i)
    print(haber.title)
    print(haber.link)
    print(haber.summary)
    print("\n")

Das System kann den angegebenen Pfad nicht finden.


1
Piyasalar bu konuşmayı gözlüyor! Fed Başkanı Powell 2024'ün yol haritasını çizecek
https://www.cnnturk.com/ekonomi/piyasalar-bu-konusmayi-gozluyor-fed-baskani-powell-2024un-yol-haritasini-cizecek-2091877
CNNTURK.COM/Hasan Selçuk TURAN-Gözler Fed Başkanı Jerome Powell'ın konuşmasında. Powell son dönemde, faiz oranlarını düşürmek için acele edilmemesi gerektiğini vurguluyor.


2
SON DAKİKA: Yerel seçime geri sayım... Cumhurbaşkanı Erdoğan Sivas'ta...
https://www.cnnturk.com/video/turkiye/son-dakika-yerel-secime-geri-sayim-cumhurbaskani-erdogan-sivasta-2091876
Cumhurbaşkanı Recep Tayyip Erdoğan, yerel seçimlere 25 gün kala Sivas'ta vatandaşlara hitap ediyor.


3
Hakkâri'nin Neyi Meşhur? Hakkâri'nin En Meşhur Yemekleri ve Alınabilecek Hediyelikler
https://www.cnnturk.com/seyahat/hakkarinin-neyi-meshur-hakkarinin-en-meshur-yemekleri-ve-alinabilecek-hediyelikler-2091871
Hem tarihi ve kültürel dokusu hem de yemekleri ile Hakkari Türkiye'nin en önemli şehirlerinden biridir. Özellikle de bal 

# coding=utf-8

"""
 n: disk sayısı,
 s: kaynak,
 t: hedef,
 b: tampon

"""


def hanoi(n, s, t, b):
    assert n > 0
    if n == 1:
        print('move', s, 'to', t)
    else:
        hanoi(n - 1, s, b, t)
        hanoi(1, s, t, b)
        hanoi(n - 1, b, t, s)


if __name__ == '__main__':
    hanoi(7, 1, 2, 1)

In [4]:
# coding=utf-8

"""
    Zamanlayıcı kullanımı
    Tarihi gösteren küçük bir uygulama
"""

import time
import os
import platform


def temizle():
    if platform.system() == 'Linux' or platform.system() == 'Darwin':
        os.system('clear')
    else:
        os.system('cls')


while True:

    zaman = time.localtime()
    yil = zaman[0]
    ay = zaman[1]
    gun = zaman[2]
    saat = zaman[3]
    dakika = zaman[4]
    saniye = zaman[5]

    time.sleep(1)
    temizle()

    print("""
        tarih: {}/{}/{}
        saat : {}:{}:{}
        """.format(gun, ay, yil, saat, dakika, saniye))


        tarih: 5/3/2024
        saat : 13:43:2
        

        tarih: 5/3/2024
        saat : 13:43:3
        

        tarih: 5/3/2024
        saat : 13:43:4
        

        tarih: 5/3/2024
        saat : 13:43:5
        

        tarih: 5/3/2024
        saat : 13:43:6
        

        tarih: 5/3/2024
        saat : 13:43:7
        

        tarih: 5/3/2024
        saat : 13:43:8
        

        tarih: 5/3/2024
        saat : 13:43:9
        

        tarih: 5/3/2024
        saat : 13:43:10
        

        tarih: 5/3/2024
        saat : 13:43:11
        

        tarih: 5/3/2024
        saat : 13:43:12
        

        tarih: 5/3/2024
        saat : 13:43:13
        


KeyboardInterrupt: 

In [26]:
import pdfplumber
import pytesseract
from PIL import Image
import pandas as pd
import io

# PDF dosyasını açın ve metinleri okuyun
with pdfplumber.open(r"C:\Users\Murathan\Desktop\MC\Murathan IM\py\Produktkatalog.pdf") as pdf:
    # Kod devam eder...

    text_data = []
    

    # PDF sayfalarını döngüye alın
    for page in pdf.pages:
        # Metni okuyun
        text = page.extract_text()
        # Metni ekrana yazdırın
        print(text)
        # Metni text_data listesine ekleyin
        text_data.append(text)

# Metinleri kontrol etmek için ekrana yazdırın
print(text_data)

ValueError: seek of closed file

In [32]:
import pdfplumber
import pytesseract
from PIL import Image
import pandas as pd
import io
from tqdm import tqdm  # tqdm is used for progress bar

# PDF dosyasını açın ve metinleri okuyun
with pdfplumber.open(r"C:\Users\Murathan\Desktop\MC\Murathan IM\py\Produktkatalog.pdf") as pdf:
    text_data = []

    # İlerleme çubuğunu oluşturun
    progress_bar = tqdm(total=len(pdf.pages), desc="Writing to TXT")

     PDF sayfalarını döngüye alın
    for page in pdf.pages:
        # Sayfadaki görüntüleri alın
        images = page.images
        
        # Görüntüler üzerinde döngüye alın
        for img in images:
            # Görüntüyü PIL formatına dönüştürün
            image = page.to_image(resolution=150)
            img_data = image.original.convert("RGB")

            # OCR işlemi yaparak metni alın
            text = pytesseract.image_to_string(img_data)

            # Metni text_data listesine ekleyin
            text_data.append(text)

        # Metni okuyun
        text = page.extract_text()
        # Metni ekrana yazdırın
        print(text)
        # Metni text_data listesine ekleyin
        text_data.append(text)

    # Metinleri bir TXT dosyasına yazın
    with open("output.txt", "w", encoding="utf-8") as txt_file:
        for text in text_data:
            txt_file.write(text + "\n")

# Metinleri kontrol etmek için ekrana yazdırın
print(text_data)

IndentationError: unexpected indent (2869004512.py, line 15)

In [34]:
import pdfplumber
import pytesseract
from tqdm import tqdm

# PDF dosyasını açın ve metinleri okuyun
with pdfplumber.open(r"C:\Users\Murathan\Desktop\MC\Murathan IM\py\Produktkatalog.pdf") as pdf:
    text_data = []

    # İlerleme çubuğunu oluşturun
    progress_bar = tqdm(total=len(pdf.pages), desc="Writing to TXT")

    # PDF sayfalarını döngüye alın
    for page in pdf.pages:
        # Sayfadaki görüntüleri alın
        images = page.images
        
        # Her bir görüntü için ilerleme çubuğunu oluşturun
        image_progress_bar = tqdm(total=len(images), desc="Processing Images", leave=False)

        # Görüntüler üzerinde döngüye alın
        for img in images:
            # Görüntü üzerinde OCR işlemi yaparak metni alın
            text = pytesseract.image_to_string(img.to_image().original.convert("RGB"))

            # Metni text_data listesine ekleyin
            text_data.append(text)

            # İlerleme çubuğunu güncelleyin
            image_progress_bar.update(1)

        # Her bir görüntü için ilerleme çubuğunu kapatın
        image_progress_bar.close()

        # Sayfadaki metni de alın
        text = page.extract_text()
        # Metni text_data listesine ekleyin
        text_data.append(text)

        # İlerleme çubuğunu güncelleyin
        progress_bar.update(1)

    # İlerleme çubuğunu kapatın
    progress_bar.close()

    # Metinleri bir TXT dosyasına yazın
    with open("output.txt", "w", encoding="utf-8") as txt_file:
        for text in text_data:
            txt_file.write(text + "\n")

# Metinleri kontrol etmek için ekrana yazdırın
print(text_data)




Writing to TXT:   0%|                                                                           | 0/72 [00:34<?, ?it/s]



AttributeError: 'dict' object has no attribute 'to_image'

In [37]:
import pdfplumber
import pytesseract
from tqdm import tqdm

# PDF dosyasını açın ve metinleri okuyun
with pdfplumber.open(r"C:\Users\Murathan\Desktop\MC\Murathan IM\py\Produktkatalog.pdf") as pdf:
    text_data = []

    # İlerleme çubuğunu oluşturun
    progress_bar = tqdm(total=len(pdf.pages), desc="Writing to TXT")

    # PDF sayfalarını döngüye alın
    for page in pdf.pages:
        # Sayfadaki görüntüleri alın
        images = page.images
        
        # Her bir görüntü için ilerleme çubuğunu oluşturun
        image_progress_bar = tqdm(total=len(images), desc="Processing Images", leave=False)

        # Görüntüler üzerinde döngüye alın
        for img_dict in images:
            # Görüntünün yukarıda belirtildiği gibi bir sözlük olduğunu unutmayın
            img = page.to_image()
            # Görüntü üzerinde OCR işlemi yaparak metni alın
            text = pytesseract.image_to_string(img["original"].convert("RGB"))

            # Metni text_data listesine ekleyin
            text_data.append(text)

            # İlerleme çubuğunu güncelleyin
            image_progress_bar.update(1)

        # Her bir görüntü için ilerleme çubuğunu kapatın
        image_progress_bar.close()

        # Sayfadaki metni de alın
        text = page.extract_text()
        # Metni text_data listesine ekleyin
        text_data.append(text)

        # İlerleme çubuğunu güncelleyin
        progress_bar.update(1)

    # İlerleme çubuğunu kapatın
    progress_bar.close()

    # Metinleri bir TXT dosyasına yazın
    with open("output.txt", "w", encoding="utf-8") as txt_file:
        for text in text_data:
            txt_file.write(text + "\n")

# Metinleri kontrol etmek için ekrana yazdırın
print(text_data)




Writing to TXT:   0%|                                                                           | 0/72 [00:09<?, ?it/s]



TypeError: 'PageImage' object is not subscriptable

In [39]:
import pdfplumber
import pytesseract
from PIL import Image
from tqdm import tqdm

# PDF dosyasını açın ve metinleri okuyun
with pdfplumber.open(r"C:\Users\Murathan\Desktop\MC\Murathan IM\py\Produktkatalog.pdf") as pdf:
    text_data = []

    # İlerleme çubuğunu oluşturun
    progress_bar = tqdm(total=len(pdf.pages), desc="Writing to TXT")

    # PDF sayfalarını döngüye alın
    for page in pdf.pages:
        # Sayfadaki görüntüleri alın
        images = page.images
        
        # Her bir görüntü için ilerleme çubuğunu oluşturun
        image_progress_bar = tqdm(total=len(images), desc="Processing Images", leave=False)

        # Görüntüler üzerinde döngüye alın
        for img_dict in images:
            # Görüntüyü PIL formatına dönüştürün
            img = page.to_image().original.convert("RGB")
            
            # Görüntü üzerinde OCR işlemi yaparak metni alın
            text = pytesseract.image_to_string(img)

            # Metni text_data listesine ekleyin
            text_data.append(text)

            # İlerleme çubuğunu güncelleyin
            image_progress_bar.update(1)

        # Her bir görüntü için ilerleme çubuğunu kapatın
        image_progress_bar.close()

        # Sayfadaki metni de alın
        text = page.extract_text()
        # Metni text_data listesine ekleyin
        text_data.append(text)

        # İlerleme çubuğunu güncelleyin
        progress_bar.update(1)

    # İlerleme çubuğunu kapatın
    progress_bar.close()

    # Metinleri bir TXT dosyasına yazın
    with open("output.txt", "w", encoding="utf-8") as txt_file:
        for text in text_data:
            txt_file.write(text + "\n")

# Metinleri kontrol etmek için ekrana yazdırın
print(text_data)



Processing Images:   0%|                                                                         | 0/2 [00:00<?, ?it/s]

                                                                                                                       

TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.